# Generating sets

Here we first define a couple of functions to provide
generating sets for the symmetric groups.

In [1]:
from itertools import combinations, permutations, product

import numpy as np

from utils import random_sp, random_tsp, phi, psi, build_curve

First the set of contiguous transpositions.

In [2]:
def nswap_generators(n):
    G = []
    eye = np.eye(n, dtype=np.uint16)
    for i in range(n):
        j = (i + 1) % n
        g = np.copy(eye)
        g[i] = eye[j]
        g[j] = eye[i]
        G.append(g)
    return G

And the set of all transpositions.

In [3]:
def lswap_generators(n):
    G = []
    eye = np.eye(n, dtype=np.uint16)
    ijs = combinations(range(n), 2)
    for i, j in ijs:
        g = np.copy(eye)
        g[i] = eye[j]
        g[j] = eye[i]
        G.append(g)
    return G

And sets of k-opt generators.

In [4]:
def k_opt_generators(n, k):
    eye = np.eye(n, dtype=np.uint16)
    all_cuts = combinations(range(n), k)
    G = []
    seen = set()
    for cuts in all_cuts:
        offset = cuts[0]
        cuts = np.array(cuts[1:]) - offset
        pieces = []
        start = 0
        for cut in cuts:
            piece = eye[start:cut]
            pieces.append((piece, piece[::-1]))
            start = cut
        last_piece = eye[start:]
        pieces.append((last_piece, last_piece[::-1]))
        reorderings = permutations(pieces)
        for reordering in reorderings:
            orderings = product((0, 1), repeat=k)
            for ordering in orderings:
                newgen = [piece[o] for piece, o in zip(reordering, ordering)]
                anewgen = np.concatenate(newgen)
                maybe_newgen = np.copy(eye)
                maybe_newgen[:offset] = anewgen[n - offset:]
                maybe_newgen[offset:] = anewgen[:n - offset]
                s = ''.join(map(str, np.ravel(maybe_newgen)))
                if s not in seen:
                    G.append(maybe_newgen)
                    seen.add(s)
    return G

We can check contiguous transpositions to solve the SP.

In [5]:
n = 20

sp1 = random_sp(n)
G = nswap_generators(n)

print(f"Unsolved: {sp1}")

solved = build_curve(sp1, G, phi)

print(f"Steps taken: {len(solved)}")
print(f"Solution: {solved[-1]}")

Unsolved: [0.46421274 0.01528442 0.77352475 0.46909602 0.61730303 0.40204241
 0.03000184 0.4744963  0.26330173 0.80658286 0.07201637 0.92621212
 0.98374375 0.50353513 0.80629306 0.98305901 0.14781505 0.40326868
 0.85251745 0.56234082]
Steps taken: 73
Solution: [0.01528442 0.03000184 0.07201637 0.14781505 0.26330173 0.40204241
 0.40326868 0.46421274 0.46909602 0.4744963  0.50353513 0.56234082
 0.61730303 0.77352475 0.80629306 0.80658286 0.85251745 0.92621212
 0.98305901 0.98374375]


Also general transpositions

In [6]:
G2 = lswap_generators(n)

solved = build_curve(sp1, G2, phi)

print(f"Steps taken: {len(solved)}")
print(f"Solution: {solved[-1]}")

Steps taken: 21
Solution: [0.01528442 0.03000184 0.07201637 0.14781505 0.26330173 0.40204241
 0.40326868 0.46421274 0.46909602 0.4744963  0.50353513 0.56234082
 0.61730303 0.77352475 0.80629306 0.80658286 0.85251745 0.92621212
 0.98305901 0.98374375]


2-opt generating set

In [7]:
G3 = k_opt_generators(n, 2)

solved = build_curve(sp1, G3, phi)

print(f"Steps taken: {len(solved)}")
print(f"(Non) Solution: {solved[-1]}")

Steps taken: 3
(Non) Solution: [0.03000184 0.4744963  0.26330173 0.80658286 0.07201637 0.40204241
 0.61730303 0.46909602 0.77352475 0.01528442 0.46421274 0.56234082
 0.85251745 0.40326868 0.14781505 0.98305901 0.80629306 0.50353513
 0.92621212 0.98374375]


3-opt generating set

In [8]:
G4 = k_opt_generators(n, 3)

solved = build_curve(sp1, G4, phi)

print(f"Steps taken: {len(solved)}")
print(f"Solution: {solved[-1]}")

Steps taken: 19
Solution: [0.01528442 0.03000184 0.07201637 0.14781505 0.26330173 0.40204241
 0.40326868 0.46421274 0.46909602 0.4744963  0.50353513 0.56234082
 0.61730303 0.77352475 0.80629306 0.80658286 0.85251745 0.92621212
 0.98305901 0.98374375]
